In [1]:
# Import dependencies
import json, os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveJsonSplitter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


True

In [2]:
all_docs = []
for subdir, dirs, files in os.walk('./car_data'):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(subdir, file)
            
            document = json.load(open(file_path))
            all_docs.append(document)
print(len(all_docs)) 

224


In [5]:
# Load them up into a vectorstore
json_splitter = RecursiveJsonSplitter(max_chunk_size=500)
docs = json_splitter.create_documents(texts=all_docs)
print(f"Created {len(docs)} chunks")
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

Created 655 chunks


In [19]:
# Prompt Template 
# https://smith.langchain.com/hub/rlm/rag-prompt

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know.
{question}
{context}
  """

prompt = PromptTemplate(
    input_variables=["context", "question"], 
    template=prompt_template,
  )

In [20]:
# Build the RAG Chain & Invoke

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#llm = Ollama(model="llama3")
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Tell me about porsche cayenne")

"I'm sorry, but the retrieved context does not contain information about the Porsche Cayenne."